In [ ]:
#THIS FUNCTION IS FOR RUNNING WITH SLURM JOB ARRAY
#(SPLITS UP JOB_ARRAY BELOW INTO EVEN MORE TASKS)
def StartSlurmJobArray(num_jobs,num_slurm_jobs, ISRUN):
    job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    if job_id==0: job_id=1
    if ISRUN==False:
        start_job=1;end_job=num_jobs+1
        return start_job,end_job
    total_elements=num_jobs #total num of variables

    job_range = total_elements // num_slurm_jobs  # Base size for each chunk
    remaining = total_elements % num_slurm_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_slurm_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_slurm_jobs - 1: 
            end_job = total_elements 
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_slurm_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()
    # if sbatch==True:
        
    start_job, end_job = get_job_range(job_id, num_slurm_jobs)
    index_adjust=start_job
    # print(f'start_job = {start_job}, end_job = {end_job}')
    if start_job==0: start_job=1
    if end_job==total_elements: end_job+=1
    return start_job,end_job
# job_id=1
# [start_slurm_job,end_slurm_job,slurm_index_adjust]=StartSlurmJobArray(num_jobs,num_slurm_jobs,ISRUN)

In [ ]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import matplotlib.lines as mlines
import xarray as xr
import os; import time
import pickle
import h5py

In [ ]:
#MAIN DIRECTORIES
mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
scratchDirectory='/home/air673/koa_scratch/'
codeDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Project_Algorithms/Tracking_Algorithms'

In [ ]:
#LOADING DATA
def GetDataDirectories(simulationNumber):
    if simulationNumber == 1:
        Directory=os.path.join(mainDirectory,'Model/cm1r20.3/run')
        res='1km'; t_res='5min'; Np_str='1e6'; Nz_str='34'
    elif simulationNumber == 2:
        Directory=scratchDirectory
        res='1km'; t_res='1min'; Np_str='50e6'; Nz_str='95'
    elif simulationNumber == 3:
        Directory=scratchDirectory
        res='250m'; t_res='1min'; Np_str='50e6'; Nz_str='95'
        
    dataDirectory = os.path.join(Directory, f"cm1out_{res}_{t_res}_{Nz_str}nz.nc")
    parcelDirectory = os.path.join(Directory,f"cm1out_pdata_{res}_{t_res}_{Np_str}np.nc")
    return dataDirectory, parcelDirectory, res,t_res,Np_str,Nz_str
    
def GetData(dataDirectory, parcelDirectory):
    dataNC = xr.open_dataset(dataDirectory, decode_timedelta=True) 
    parcelNC = xr.open_dataset(parcelDirectory, decode_timedelta=True) 
    return dataNC,parcelNC

def SubsetDataVars(dataNC):
    varList = ["thflux", "qvflux", "tsk", "cape", 
               "cin", "lcl", "lfc", "th",
               "prs", "rho", "qv", "qc",
               "qr", "qi", "qs","qg", 
               "buoyancy", "uinterp", "vinterp", "winterp",]
    
    varList += ["ptb_hadv", "ptb_vadv", "ptb_hidiff", "ptb_vidiff",
                "ptb_hturb", "ptb_vturb", "ptb_mp", "ptb_rdamp", 
                "ptb_rad", "ptb_div", "ptb_diss",]
    
    varList += ["qvb_hadv", "qvb_vadv", "qvb_hidiff", "qvb_vidiff", 
                "qvb_hturb", "qvb_vturb", "qvb_mp",]
    
    varList += ["wb_hadv", "wb_vadv", "wb_hidiff", "wb_vidiff",
                "wb_hturb", "wb_vturb", "wb_pgrad", "wb_rdamp", "wb_buoy",]

    return dataNC[varList]

[dataDirectory,parcelDirectory, res,t_res,Np_str,Nz_str] = GetDataDirectories(simulationNumber=1)
[data1,parcel1] = GetData(dataDirectory, parcelDirectory)

In [ ]:
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

In [ ]:
#########################################

In [ ]:
############################################################################################
times=data1['time'].values/(1e9 * 60); times=times.astype(float);
minutes=1/times[1] #1 / minutes per timestep = timesteps per minute
kms=np.argmax(data1['xh'].values-data1['xh'][0].values >= 1)

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [ ]:
import sys
import os

# Global variable to store original stdout
_original_stdout = sys.stdout

def BlockPrint():
    """Suppress all print() output."""
    global _original_stdout
    sys.stdout = open(os.devnull, 'w')

def RestorePrint():
    """Restore print() output."""
    global _original_stdout
    sys.stdout.close()
    sys.stdout = _original_stdout

# NO_PRINT=False
NO_PRINT=True

In [ ]:
##############################
# SETUP

In [ ]:
################################
#JOB ARRAY SETUP
################################
#*#*
# how many total jobs are being run? i.e. array=1-100 ==> num_jobs=100
if '1e6' in Np_str:
    num_jobs=60 #1M parcels
    num_slurm_jobs=10
if '50e6' in Np_str:
    num_jobs=200 #50M parcels
    num_slurm_jobs=60
##############################

In [ ]:
##############################################
#DATA LOADING FUNCTIONS

In [ ]:
#JOB ARRAY SETUP
def StartJobArray(job_id,num_jobs):
    total_elements=len(parcel1['xh']) #total num of variables

    if num_jobs >= total_elements:
        raise ValueError("Number of jobs cannot be greater than or equal to total elements.")
    
    job_range = total_elements // num_jobs  # Base size for each chunk
    remaining = total_elements % num_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_jobs - 1: 
            end_job = total_elements #- 1
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()

    # if sbatch==True:
    #     job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    #     if job_id==0: job_id=1
        
    start_job, end_job = get_job_range(job_id, num_jobs)
    index_adjust=start_job
    # print(f'start_job = {start_job}, end_job = {end_job}')
    return start_job,end_job,index_adjust
# job_id=1
# [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)

In [ ]:
#SUBSETTING PARCEL DATA
def GetData(parcel1,start_job,end_job):
    parcel=parcel1.isel(xh=slice(start_job,end_job))
    return parcel
# parcel=GetData(parcel1,start_job,end_job)

In [ ]:
def GetSpatialData(start_job,end_job):
    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/OUTPUT/'
    open_file=dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5'
    with h5py.File(open_file, 'r') as f:
        parcel_z = f['z'][:,start_job:end_job]
        parcel_x = f['x'][:,start_job:end_job] 

        parcel_u = f['u'][:,start_job:end_job] 
        parcel_w = f['w'][:,start_job:end_job] 
        
        # Load the dataset by its name
        Z = f['Z'][:,start_job:end_job]
        Y = f['Y'][:,start_job:end_job]
        X = f['X'][:,start_job:end_job]
        W = f['W'][:,start_job:end_job]

    return parcel_z,parcel_x,parcel_u,parcel_w,Z,Y,X,W
# [parcel_z,parcel_x,parcel_u,parcel_w,Z,Y,X,W] = GetSpatialData(start_job,end_job)

In [ ]:
def GetLFCData(start_job,end_job):
    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/OUTPUT/'
    open_file=dir2+f'lagrangian_LFC_LCL_binary_array_{res}_{t_res}_{Np_str}.h5'
    with h5py.File(open_file, 'r') as f:
        # Load the dataset by its name
        LFC = f['LFC'][:,start_job:end_job]
        LCL = f['LCL'][:,start_job:end_job]
    return LFC,LCL
# LFC,LCL = GetLFCData(start_job,end_job)

In [ ]:
#################################
#LOADING CL MAXS FROM CL TRACKING ALGORITHM
# File path
file_path = f'/mnt/lustre/koa/koastore/torri_group/air_directory/' \
            f'DCI-Project/Project_Algorithms/Tracking_Algorithms/' \
            f'CL_Tracking_Out/' \
            f'whereCL_{res}_{t_res}_ALL_CLS.h5'

# Open dataset (as it's valid NetCDF)
whereCL = xr.open_dataset(file_path, engine='netcdf4')['maxconv_x']

def Get_Conv_X(t,z,y):
    # Conv_X_Max=whereCL[t,z,y,:].data
    Conv_X_Max=whereCL.isel(time=t,z=z,y=y).data
    return Conv_X_Max

In [ ]:
#################################################################

In [ ]:
#Updated Lagrangian Tracking Algorithm

#Algorithm Steps:
#(1) Find the first time a parcel is above the LFC:
#(2) First check if the parcel ascends (w>=0.1) for another 20 minutes
#(3) If so, find first time, the parcel slows down (w<0.1)
#(4) If that time is when the parcel is above 750m, save it, "forget", and move on to next parcel
#(5) If that time is when the parcel is below 750m, check if it is within 2km of the CL_Max found from the CL Tracking Algorithm
#(6) If the parcel is near the CL, store in, otherwise save it, "forget", and move on to next parcel
#(7) Continue to next parcel

#(Also, if during, traceback, the parcel escapes the x or z boundary, "forget" parcel, and move on)

In [ ]:
#Numerical Settings
Nt=len(data1['time']) 
dt=times[1]*60
#Height parcel must be below to be counted
CLmaxheight=750 #750m
#BL slow-down-threshold
w_thresh=0.1
def GetNp(parcel):
    Np=len(parcel['xh'])
    # ascend_lst=[]
    return Np
# Np=GetNp(parcel)

In [ ]:
# if ((x + dt*u)==0) or ((z + dt*w)==0)
# u=u[t,Z[t,p],Y[t,p],X[t,p]]; W=W[t,p]
# [u[t,Z[t,p],Y[t,p],X[t,p]] for t in time_arr] >np.max(data['xf'].values) or < np.min(data['xf'].values)
# similarly for w
################################################################################################################
#BOUNDARY-ESCAPE CONDITION
xmin=np.min(data1['xf'].values)*1e3
xmax=np.max(data1['xf'].values)*1e3
zmin=np.min(data1['zf'].values)*1e3
zmax=np.max(data1['zf'].values)*1e3

def check_boundary(p,where_BL,above_LFC):
    time_arr=np.arange(where_BL,above_LFC)

    def get_x(t,p):
        # return parcel['x'][t,p].item()
        return parcel_x[t,p] 
    def get_u(t,p):
        # return data['uinterp'].isel(time=t,zh=Z[t,p],yh=Y[t,p],xh=X[t,p]).item() #TESTING
        # return parcel['u'][t,p].item() 
        return parcel_u[t,p]
    def get_z(t,p):
        # return parcel['z'][t,p].item()
        return parcel_z[t,p]
    def get_w(t,p):
        # return data['winterp'].isel(time=t,zh=Z[t,p],yh=Y[t,p],xh=X[t,p]).item()
        # return parcel['w'][t,p].item()
        return parcel_w[t,p]
        

    # x_tend = [get_x(t, p) + dt * get_u(t, z, y, x)   #THIS IS OLD, LESS IDEAL
    #       for (t, z, y, x) in zip(time_arr, Z[time_arr, p], Y[time_arr, p], X[time_arr, p])] 
    # z_tend = [get_z(t, p) + dt * get_w(t, z, y, x)  
    #       for (t, z, y, x) in zip(time_arr, Z[time_arr, p], Y[time_arr, p], X[time_arr, p])] 
    
    x_tend = [get_x(t, p) + dt * get_u(t,p)   
          for (t, z, y, x) in zip(time_arr, Z[time_arr, p], Y[time_arr, p], X[time_arr, p])] 
    z_tend = [get_z(t, p) + dt * get_w(t,p)  
          for (t, z, y, x) in zip(time_arr, Z[time_arr, p], Y[time_arr, p], X[time_arr, p])] 

    x_bound=any(val < xmin or val > xmax for val in x_tend)*1
    z_bound=any(val < zmin or val > zmax for val in z_tend)*1

    out=(x_bound,z_bound)
    if out[0]==1:
        print(f'parcel {p} crossed x-boundary between t={where_BL} and t={above_LFC}')
    elif out[1]==1:
        print(f'parcel {p} crossed z-boundary between t={where_BL} and t={above_LFC}')
    return out
#############################################################################################################

In [ ]:
#Initialize Output Storage Vector
def InitializeData(Np):
    out_arr=np.zeros((Np,3),dtype=np.int32) 
    save_arr=np.zeros((Np,3),dtype=np.int32) #This one is for saving continued-ascent, slow-below-750m parcels that are not with 2 km of CL
    save2_arr=np.zeros((Np,3),dtype=np.int32) #This one is for saving continued-ascent, slow-above-750m parcels
    return out_arr,save_arr,save2_arr
# [out_arr,save_arr,save2_arr]=InitializeData(Np)

In [ ]:
#############################################################################################################
#The Algorithm

In [ ]:
def ParcelTracking(Np,W,LFC,parcel_z,Z,Y,X,out_arr,save_arr,save2_arr):
    #1--------------Looping over each parcel
    for count,p in enumerate(np.arange(Np)): 
        if np.mod(p,5e4)==0: print(f'current parcel: {p}/{Np}')
        
        W_p = W[:,p]
        LFC_p = LFC[:,p] 
       
        #----FIND WHERE PARCEL IS ABOVE LFC----
        indices = np.where(LFC_p == 1)[0]; above_LFC = indices[0] if indices.size > 0 else -999; #FIRST TIME ABOVE LFC
        if above_LFC ==-999:
            # print(f'parcel {p} never above LFC')
            continue #if the parcel is never above the LFC, skip the parcel
        
        #----CHECK IF ASCENDS FOR >= 20 minutes AFTER LFC----
        ascend_array=W_p[above_LFC+1:]
        indices=np.where(ascend_array==0)[0]; ascend_stop=indices[0] if indices.size > 0 else 10000; #location of where parcel stops ascending (labeled 10000 to mark for future analysis)
        # ascend_lst.append(ascend_stop) #(also store for histogram)
        if ascend_stop>=20*minutes:
        
            #----FIND THE FIRST TIME W_p<=w_thresh----
            indices=np.where(W_p[0:above_LFC]<w_thresh)[0]
            where_BL=indices[-1] if indices.size > 0 else -999 #FIRST PRIOR TIME W<0.1 (IN THE BL) (ADDED 1 TO GET TIME RIGHT AFTER INTERACTION)
            if where_BL ==-999:
                # print(f'parcel {p} w is never below threshold prior to t={above_LFC}')
                continue #if the parcel never slows down backwards in time (unlikely), skip the parcel
                
            #check for boundary escapes
            ################################
            future_location=check_boundary(p,where_BL,above_LFC)
            if (future_location[0]+future_location[1]>=1): continue #if parcel crosses boundary, skips current parcel
            ################################
            
            #----CHECK IF PARCEL SLOWED DOWN LOW ENOUGH----
            if parcel_z[where_BL,p]<=CLmaxheight: #PARCEL MUST BE BELOW 750m WHEN CONTACTING CL #***
            # if LCL[where_BL,p]==0: #PARCEL MUST BE BELOW LCL WHEN CONTACTING CL (not recommended)
        
                #----CHECK IF CL IS WITHIN 2km----
                #Find the CL-max x-location
                t=where_BL; z=Z[where_BL,p]; y=Y[where_BL,p]; x=X[where_BL,p]
                CONV_X=Get_Conv_X(t,z,y)
                within_CL=np.any(np.isin(CONV_X, np.arange(x-2*kms,x+3*kms)))
                
                if within_CL==True:
                    #save X's (t,p) 
                    print(f'Parcel {p} is success at time {where_BL}')
                    out_arr[p,0]=p
                    out_arr[p,1]=where_BL
                    out_arr[p,2]=above_LFC 
                else: #continued-ascent, slow-below-750m parcels that are not with 2 km of CL
                    #SAVE PARCEL
                    # print(f'Parcel {p} not near CL at t={where_BL}')
                    save_arr[p,0]=p
                    save_arr[p,1]=where_BL
                    save_arr[p,2]=above_LFC 
        
            else: #continued-ascent, slow-above-750m parcels
                #SAVE PARCEL
                # print(f'Parcel {p} above {CLmaxheight}m at t={where_BL}')
                save2_arr[p,0]=p
                save2_arr[p,1]=where_BL
                save2_arr[p,2]=above_LFC         
                
            #END OF LOOP, THEN WE MOVE ON TO NEXT PARCEL p
    return out_arr,save_arr,save2_arr
# [out_arr,save_arr,save2_arr]=ParcelTracking(Np,W,LFC,parcel_z,Z,Y,X,out_arr,save_arr,save2_arr)

In [ ]:
def CorrectParcelID(out_arr,save_arr,save2_arr,index_adjust):
    #CORRECTING DATA PARCEL ID BASED ON JOB NUMBER
    #####################################################
    out_arr[np.where(np.any(out_arr != 0, axis=1))[0],0]+=index_adjust #*needed for job array*+=index_adjust #*needed for job array*
    save_arr[np.where(np.any(save_arr != 0, axis=1))[0],0]+=index_adjust #*needed for job array*+=index_adjust #*needed for job array*
    save2_arr[np.where(np.any(save2_arr != 0, axis=1))[0],0]+=index_adjust #*needed for job array*+=index_adjust #*needed for job array*
    return out_arr,save_arr,save2_arr
# [out_arr,save_arr,save2_arr]=CorrectParcelID(out_arr,save_arr,save2_arr,index_adjust)

In [ ]:
#REMOVING BLANK ROWS
def RemoveZeroRows(arr):
    arr = arr[~np.all(arr == 0, axis=1)]
    return arr
# out_arr=RemoveZeroRows(out_arr);save_arr=RemoveZeroRows(save_arr);save2_arr=RemoveZeroRows(save2_arr)

In [ ]:
def SaveOutFile(out_arr,save_arr,save2_arr,job_id):
    # #Storing output and save data
    #USES H5 FILE
    ###################################################################################################################################
    out_file = dir + f'Project_Algorithms/Tracking_Algorithms/trackout/parcel_tracking_{res}_{t_res}_{Np_str}_{job_id}.h5'
    with h5py.File(out_file, 'w') as hf:
        hf.create_dataset('out_arr', data=out_arr, compression="gzip")
        hf.create_dataset('save_arr', data=save_arr, compression="gzip")
        hf.create_dataset('save2_arr', data=save2_arr, compression="gzip")
# SaveOutFile(out_arr,save_arr,save2_arr,job_id)

In [ ]:
##############################################
#RUNNING

In [ ]:
[start_slurm_job,end_slurm_job]=StartSlurmJobArray(num_jobs=num_jobs,num_slurm_jobs=num_slurm_jobs,ISRUN=True) #if ISRUN is False, then will not run using slurm_job_array
print(f"Running on Slurm_Jobs for Slurm_Job_Ids: {(start_slurm_job,end_slurm_job-1)}")
StartTime = time.time()
job_id_list=np.arange(start_slurm_job,end_slurm_job)
for job_id in job_id_list:
    if job_id % 1 == 0: print(f'current job_id = {job_id}')
    [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)
    

    #SLICING DATA
    parcel=GetData(parcel1,start_job,end_job)

    #GETTING REQUIRED DATA
    [parcel_z,parcel_x,parcel_u,parcel_w,Z,Y,X,W] = GetSpatialData(start_job,end_job)
    LFC,LCL = GetLFCData(start_job,end_job)

    #INITIALIZING DATA
    Np=GetNp(parcel)
    [out_arr,save_arr,save2_arr]=InitializeData(Np)

    #RUNNING ALGORITHM
    start_time = time.time()
    if NO_PRINT==True: BlockPrint()
    [out_arr,save_arr,save2_arr]=ParcelTracking(Np,W,LFC,parcel_z,Z,Y,X,out_arr,save_arr,save2_arr)
    if NO_PRINT==True: RestorePrint()
    end_time = time.time(); elapsed_time = end_time - start_time; print(f"Elapsed Time: {elapsed_time} seconds")  

    #CORRECTING PARCEL ID FOR JOBARRAY
    [out_arr,save_arr,save2_arr]=CorrectParcelID(out_arr,save_arr,save2_arr,index_adjust)

    #REMOVING BLANK ROWS FROMRESULTS
    out_arr=RemoveZeroRows(out_arr);save_arr=RemoveZeroRows(save_arr);save2_arr=RemoveZeroRows(save2_arr)

    #SAVING
    SaveOutFile(out_arr,save_arr,save2_arr,job_id)
EndTime = time.time(); ElapsedTime = EndTime - StartTime; print(f"Total Elapsed Time: {ElapsedTime} seconds")      

In [ ]:
############################################################
#Run after finishing job_array
recombine=False #KEEP FALSE WHEN JOB_ARRAY IS RUNNING
recombine=True 

In [ ]:
def Recombine(num_jobs):
    var_names = ['out_arr', 'save_arr', 'save2_arr']
    recombined_arrays = {}  # Store final arrays here

    def MakeUnique(arr):
        return np.unique(arr, axis=0)
    def get_total_count(var_name):
        return sum(
            h5py.File(dir + f'Project_Algorithms/Tracking_Algorithms/trackout/parcel_tracking_{res}_{t_res}_{Np_str}_{job_id}.h5', 'r')[var_name].shape[0]
            for job_id in range(1, num_jobs + 1)
        )

    def load_file(job_id, var_name):
        in_file = dir + f'Project_Algorithms/Tracking_Algorithms/trackout/parcel_tracking_{res}_{t_res}_{Np_str}_{job_id}.h5'
        with h5py.File(in_file, 'r') as hf:
            arr = hf[var_name][:]
        return arr

    # Preallocate arrays
    for var_name in var_names:
        total_count = get_total_count(var_name)
        recombined_arrays[var_name] = np.zeros((total_count, 3), dtype=np.int32)

    # Fill arrays
    for var_name in var_names:
        print(f"Combining data for {var_name}")
        left_ind = 0
        for job_id in range(1, num_jobs + 1):
            # if job_id % 10 == 0: print(f"{var_name}: processing job {job_id}")
            arr = load_file(job_id, var_name)
            n_rows = arr.shape[0]; right_ind = left_ind + n_rows
            recombined_arrays[var_name][left_ind:right_ind, :] = arr
            left_ind = right_ind

    #Make Unique
    for var_name in var_names:
        recombined_arrays[var_name]=MakeUnique(recombined_arrays[var_name])
        
    # Write to file
    out_file = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/parcel_tracking_{res}_{t_res}_{Np_str}.h5' #*#*
    with h5py.File(out_file, 'w') as hf:
        for var_name in var_names:

            hf.create_dataset(var_name, data=recombined_arrays[var_name], compression="gzip")

In [ ]:
if recombine==True:
    Recombine(num_jobs)

In [ ]:
#######################################################

In [ ]:
#LOADING BACK IN
def load_file():
    in_file=dir+f'Project_Algorithms/Tracking_Algorithms/parcel_tracking_{res}_{t_res}_{Np_str}.h5'
    with h5py.File(in_file, 'r') as hf:
        out_arr=hf['out_arr'][:]
        save_arr=hf['save_arr'][:]
        save2_arr=hf['save2_arr'][:]
    return out_arr,save_arr,save2_arr
[out_arr,save_arr,save2_arr]=load_file()

In [ ]:
#######################################################
#TESTING

In [ ]:
# #TESTING
# out_file = dir + f'Project_Algorithms/Tracking_Algorithms/parcel_tracking_{res}_{t_res}_{Np_str}.h5'
# with h5py.File(out_file, 'r') as hf:
#     test=hf['out_arr'][0:232]
# np.all(test==out_arr)

In [ ]:
#TESTING (ADDITIONAL TRACKED PARCELS FROM JOB 60
# # ind=0
# ind+=1;print(ind)
# ind_arr = out_arr[ind]
# p=ind_arr[0]-index_adjust
# t1=ind_arr[1]-10
# t2=ind_arr[1]+10
# plt.plot(parcel_z[t1:t2,p])
# plt.xlabel('t');plt.ylabel('z')